<a href="https://colab.research.google.com/github/Itsuki-Hamano123/auto_ml/blob/master/autogluon/experiments/bigwine_table_cls.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install scikit-learn==0.23.1
%pip install --upgrade mxnet-cu101
%pip install autogluon

  Using cached https://files.pythonhosted.org/packages/d9/3a/eb8d7bbe28f4787d140bb9df685b7d5bf6115c0e2a969def4027144e98b6/scikit_learn-0.23.1-cp36-cp36m-manylinux1_x86_64.whl
ERROR: autogluon 0.0.12 has requirement scikit-learn<0.23,>=0.22.0, but you'll have scikit-learn 0.23.1 which is incompatible.
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1
Requirement already up-to-date: mxnet-cu101 in /usr/local/lib/python3.6/dist-packages (1.6.0)
  Using cached https://files.pythonhosted.org/packages/5e/d8/312e03adf4c78663e17d802fe2440072376fee46cada1404f1727ed77a32/scikit_learn-0.22.2.post1-cp36-cp36m-manylinux1_x86_64.whl
  Found existing installation: scikit-learn 0.23.1
    Uninstalling scikit-learn-0.23.1:
      Successfully uninstalled scikit-learn-0.23.1


In [2]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


In [3]:
%cd /gdrive/My Drive/機械学習練習/AutoML/AutoGluon/experiments
%ls

/gdrive/My Drive/機械学習練習/AutoML/AutoGluon/experiments
AutogluonModels/         dask-worker-space/  sample_table_cls.ipynb
bigwine_table_cls.ipynb  dataset/
covtype_table_cls.ipynb  models/


# Bigwineデータセット予測モデルを作成

In [38]:
import datetime, os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import autogluon as ag
from autogluon import TabularPrediction as task
from autogluon.task.tabular_prediction import TabularDataset
from sklearn.metrics import classification_report, confusion_matrix

## データの読み込み

In [13]:
SEED = 2

DATA_DIR = 'dataset/big_wine'
TRAIN_CSV = 'preprocessed_train.csv'
TEST_CSV = 'preprocessed_test.csv'
TARGET_COLUM = 'target'

train_data = TabularDataset(file_path=os.path.join(DATA_DIR ,TRAIN_CSV ))
test_data = TabularDataset(file_path=os.path.join(DATA_DIR ,TEST_CSV ))
print('train_data:{}'.format(train_data.shape))
print('test_data:{}'.format(test_data.shape))
train_data.head()

Loaded data from: dataset/big_wine/preprocessed_train.csv | Columns = 14 / 14 | Rows = 142 -> 142
Loaded data from: dataset/big_wine/preprocessed_test.csv | Columns = 14 / 14 | Rows = 36 -> 36


train_data:(142, 14)
test_data:(36, 14)


,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
0,-0.109781,0.986826,-0.055207,-0.270719,0.400136,-1.424148,-1.348592,0.315056,-1.130436,0.070159,-1.273811,-1.236580,-0.299237,2
1,0.825554,2.939229,0.299518,0.315289,-0.366751,-0.974682,-1.437680,1.272017,-0.922865,1.169546,-1.454575,-1.250822,-0.099601,2
2,1.336870,-0.761462,-0.161625,-0.768827,-0.366751,-0.168741,0.363881,-0.801398,-0.040686,-0.057042,0.940544,0.287355,1.558908,0
3,0.663429,0.214739,1.150857,1.487306,0.330419,-1.176167,-1.210010,0.235309,-0.092579,1.605666,-1.002666,-1.165368,-0.068888,2
4,0.314237,-0.548473,-0.800129,-0.710226,-0.436468,0.141236,0.126313,-0.721652,-0.421234,-0.534049,0.262680,0.216143,1.574264,0


## ベストなモデルを探索する

### 実験の詳細を指定

In [15]:
# 実験の最大許容時間の設定
time_limits = 300  #@param {type: "number"}

# 訓練後のモデル出力先
output_directory = 'models'
model_name = 'bigwine_' + datetime.datetime.now().strftime('%Y%m%d%H%M%S')
output_model_path = os.path.join(output_directory, model_name)

In [16]:
predictor = task.fit(train_data=train_data, tuning_data=test_data, label=TARGET_COLUM,
                     auto_stack=True, # アンサンブルの戦略も自動で行ってほしい場合
                     #num_bagging_folds=5, stack_ensemble_levels=1, #決め打ちしたい場合
                     time_limits=time_limits)

No output_directory specified. Models will be saved in: AutogluonModels/ag-20200725_144843/
Beginning AutoGluon training ... Time limit = 300s
AutoGluon will save models to AutogluonModels/ag-20200725_144843/
AutoGluon Version:  0.0.12
Train Data Rows:    142
Train Data Columns: 14
Tuning Data Rows:    36
Tuning Data Columns: 14
Preprocessing data ...
Here are the 3 unique label values in your data:  [2, 0, 1]
AutoGluon infers your prediction problem is: multiclass  (because dtype of label-column == int, but few unique label-values observed).
If this is wrong, please specify `problem_type` argument in fit() instead (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])

Train Data Class Count: 3
NumExpr defaulting to 2 threads.
Feature Generator processed 178 data points with 13 features
Original Features (raw dtypes):
	float64 features: 13
Original Features (inferred dtypes):
	float features: 13
Generated Features (special dtypes):
Final Features (raw dtypes)

### 結果の確認

In [17]:
results = predictor.fit_summary()

*** Summary of fit() ***
Estimated performance of each model:
                                    model  score_val  pred_time_val    fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer
0                 weighted_ensemble_k0_l1   0.994382       1.275062  152.868117                0.001194           0.538868            1       True
1          NeuralNetClassifier_STACKER_l0   0.988764       0.086104  145.920906                0.086104         145.920906            0       True
2     ExtraTreesClassifierEntr_STACKER_l0   0.988764       1.086987    6.406035                1.086987           6.406035            0       True
3     ExtraTreesClassifierGini_STACKER_l0   0.988764       1.187764    6.408342                1.187764           6.408342            0       True
4     LightGBMClassifierCustom_STACKER_l0   0.983146       0.064371    4.261234                0.064371           4.261234            0       True
5   RandomForestClassifierGini_STACKER_l0   0.983146    

In [37]:
best_model_name = predictor.get_model_best()
y_pred= predictor.predict(model=best_model_name, dataset=test_data.drop(columns=TARGET_COLUM))
y_true = test_data[TARGET_COLUM].values 

class_names =[]
class_num = 3
for i in range(class_num):
    class_names.append('class_'+str(i+1))

# 分類器用のレポート出力
report =classification_report(y_true=y_true, y_pred=y_pred, target_names=class_names)
print(report)

              precision    recall  f1-score   support

     class_1       1.00      1.00      1.00         7
     class_2       1.00      1.00      1.00        17
     class_3       1.00      1.00      1.00        12

    accuracy                           1.00        36
   macro avg       1.00      1.00      1.00        36
weighted avg       1.00      1.00      1.00        36



In [46]:
# 混同行列を表示
print(confusion_matrix(y_true=y_true, y_pred=y_pred))

[[ 7  0  0]
 [ 0 17  0]
 [ 0  0 12]]
